In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:24<00:00, 1079139.61it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 125955.14it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:07<00:00, 555835.76it/s] 


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
# noinspection PyShadowingNames
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
# noinspection PyShadowingNames
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # enable eval mode
    model.eval()
    test_loss, correct = 0, 0
    # freeze gradient descent
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.157875  [   64/60000]
loss: 1.160280  [ 6464/60000]
loss: 0.976364  [12864/60000]
loss: 1.113079  [19264/60000]
loss: 0.978371  [25664/60000]
loss: 1.019777  [32064/60000]
loss: 1.053219  [38464/60000]
loss: 0.990824  [44864/60000]
loss: 1.031162  [51264/60000]
loss: 0.967790  [57664/60000]
Test Error: 
 Accuracy: 66.1%, Avg loss: 0.980157 

Epoch 2
-------------------------------
loss: 1.039282  [   64/60000]
loss: 1.063411  [ 6464/60000]
loss: 0.860884  [12864/60000]
loss: 1.020649  [19264/60000]
loss: 0.892471  [25664/60000]
loss: 0.926080  [32064/60000]
loss: 0.977629  [38464/60000]
loss: 0.914420  [44864/60000]
loss: 0.952958  [51264/60000]
loss: 0.903485  [57664/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 0.907675 

Epoch 3
-------------------------------
loss: 0.950832  [   64/60000]
loss: 0.995803  [ 6464/60000]
loss: 0.777584  [12864/60000]
loss: 0.955012  [19264/60000]
loss: 0.834814  [25664/60000]
loss: 0.856665  [32064/600

# pytorch 保存模型

在PyTorch中，保存模型通常有两种主要方式：保存整个模型和保存模型的状态字典（state dictionary）

- ## 保存整个模型

    保存整个模型意味着将模型的结构和参数一起保存到一个文件中。这种方法简单直观，但缺点是保存的文件可能会包含一些不必要的信息（如优化器状态），并且加载模型时需要有相同的模型类定义
    
    ```python
    # 保存模型
    torch.save(model, 'model.pth')
  
    # 加载模型
    model = torch.load('model.pth')
    model.eval()  # 设置为评估模式
    ```

- ## 保存状态字典
    
    状态字典包含了模型的参数，不包含模型的结构。这种方法更加灵活，可以轻松地在不同的模型结构之间迁移参数
    
    ```python
    # 保存模型的状态字典
    torch.save(model.state_dict(), 'model_state_dict.pth')
    
    # 加载模型的状态字典
    model = MyModel()  # 需要先实例化模型
    model.load_state_dict(torch.load('model_state_dict.pth'))
    model.eval()  # 设置为评估模式
    ```
  
- ## 加载模型

    加载模型时，可以指定设备，以确保模型参数在正确的设备上
    
    ```python
    model.load_state_dict(torch.load('model_state_dict.pth', map_location=torch.device('cpu')))
    ```


In [14]:
torch.save(model.state_dict(), "./model/fashion_mnist.pth")
print("Saved PyTorch Model State to ./model/fashion_mnist.pth")

Saved PyTorch Model State to ./model/fashion_mnist.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("./model/fashion_mnist.pth"))